In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG

ImportError: cannot import name 'VGG8' from 'tensorflow.keras.applications' (c:\Users\juliu\.conda\lib\site-packages\keras\api\_v2\keras\applications\__init__.py)

In [4]:
class VeggieVision:
    def __init__(self, base_model, input_shape=(224, 224, 3)):
        # Set parameters
        self.input_shape = input_shape
        self.base_model = base_model
        self.model = None
        self.datagen = None

        # Build the model
        self.build_model()

    def build_model(self):
        # Freeze the base model layers initially
        self.base_model.trainable = False

       # 2. Build the Model
        self.model = models.Sequential()

        # Add the base model (MobileNetV2)
        self.model.add(self.base_model)

        self.model.add(layers.Flatten())

        # Add an additional Dense layer with 128 neurons and ReLU activation
        self.model.add(layers.Dense(1024, activation='relu'))


        self.model.add(layers.Dense(256, activation = "relu"))


        #   Final Dense layer with a single neuron for regression (predicting weight)
        self.model.add(layers.Dense(1, activation='linear'))
        

    def augment_data(self, X_train):
        # 5. Data Augmentation
        # Create an ImageDataGenerator for real-time data augmentation
        self.datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=180,         # Rotate images by up to 90 degrees
            width_shift_range=0.1,     # Shift image horizontally by up to 20% of the width
            height_shift_range=0.1,    # Shift image vertically by up to 20% of the height
            shear_range=0.1,           # Shear the image by up to 10%
            zoom_range=0.1,            # Zoom into the image by up to 20%
            horizontal_flip=True,      # Randomly flip images horizontally
            fill_mode='nearest'        # Fill in missing pixels with the nearest filled value
        )

        # Fit the ImageDataGenerator on the training data
        self.datagen.fit(X_train)

    def fit(self, X, y, batch_size=8, epochs=20, learning_rate=1e-4, validation_split=0.2, fine_tune=False):
        # 4. Split Data into Training and Validation Sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)

        # Augment the data
        self.augment_data(X_train)

        # Compile the model with the provided learning rate
        self.model.compile(optimizer=Adam(learning_rate=learning_rate), 
                           loss='mean_squared_error', 
                           metrics=['mae'])

        # 6. Train the Model
        # Use the data generator to augment and train the model
        history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                 validation_data=(X_val, y_val), 
                                 epochs=epochs)

        # Evaluate the model on the validation set
        test_loss, test_mae = self.model.evaluate(X_val, y_val)
        print(f'Test MAE after initial training: {test_mae}')

        # 8. Fine-tuning the model (if requested)
        if fine_tune:
            self.fine_tune(X_train, y_train, X_val, y_val, batch_size, epochs, learning_rate)

        return history

    def fine_tune(self, X_train, y_train, X_val, y_val, batch_size=32, epochs=10, fine_tune_learning_rate=1e-5):
        # Unfreeze the base MobileNetV2 model's layers
        self.base_model.trainable = True

        # Compile with a lower learning rate for fine-tuning
        self.model.compile(optimizer=Adam(learning_rate=fine_tune_learning_rate), 
                           loss='mean_squared_error', 
                           metrics=['mae'])

        # Fine-tune the model with augmented data
        fine_tune_history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                           validation_data=(X_val, y_val), 
                                           epochs=epochs)

        # Evaluate after fine-tuning
        test_loss, test_mae = self.model.evaluate(X_val, y_val)
        print(f'Test MAE after fine-tuning: {test_mae}')

        return fine_tune_history
    
    def evaluate(self, X, y):
        test_loss, test_mae = self.model.evaluate(X, y)
        print(f'MAE: {test_mae}')
        return test_loss, test_mae


In [5]:
X = np.load("../data/processed_data/X.npy")
y = np.load("../data/processed_data/y.npy")

In [6]:
base_model = VGG8(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggiev_bm = VeggieVision(base_model)

: 

In [ ]:
veggiev_bm.fit(X,y, learning_rate = 1e-3)

Epoch 1/20
73/73 [==============================] - ETA: 0s - loss: 15977.0586 - mae: 56.4971